In [5]:
import os
import gzip
import pandas as pd

# Contexto

Neste case fictício, você como o novo Analista de Dados tem uma demanda do time de epidemiologia da Nexodata de entender quais são as regiões pelo Brasil, nas granularidade de Cidade, que apresentaram o maior crescimento na taxa de mortalidade (devido a COVID-19) no primeiro trimestre de 2021 e quais as características em comum dessas cidades que justificam essa taxa de mortalidade crescente que se destaca do restante. O time que trouxe a demanda, é um time técnico em saúde, mas não em dados, então cabe a você pensar em estratégias para trabalhar os dados do dataset disponibilizado afim de solucionar o problema.

# Obejtivo

Com as bases disponibilizadas, o objetivo é analisar quais são as regiões do Brasil, por cidade, que apresentam maior crescimento na taxa de mortalidade por COVID-19 no primeiro trimestre de 2021 (1o TRI 2021). Também deve-se verificar quais características em comum as cidades com maiores taxa de mortalidade que esclareçam/justifiquem tal comportamento.

# Visualização dos tipos de dados

In [20]:
print(os.listdir())
lista_arquivos = []

for arquivo in os.listdir():
    if '.gz' in arquivo:
        #print(arquivo)
        lista_arquivos.append(arquivo)
        
for arquivo in lista_arquivos:
    df = pd.read_csv(arquivo, compression='gzip')
    print(df.info())

['.ipynb_checkpoints', 'boletim.csv.gz', 'caso.csv.gz', 'caso_full.csv.gz', 'Link.txt', 'obito_cartorio.csv.gz', 'Untitled.ipynb']
boletim.csv.gz
caso.csv.gz
caso_full.csv.gz
obito_cartorio.csv.gz
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20827 entries, 0 to 20826
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    20827 non-null  object
 1   notes   6565 non-null   object
 2   state   20827 non-null  object
 3   url     20827 non-null  object
dtypes: object(4)
memory usage: 651.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838003 entries, 0 to 2838002
Data columns (total 13 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   date                            object 
 1   state                           object 
 2   city                            object 
 3   place_type                      object 
 4   confirmed                       int64  
 5  

- boletim.csv.gz: possuí os url's dos boletins que originaram os dados de casos de COVID-19 em cada estado. <br>
- caso.csv.gz: possuí informações resumidas de casos de COVID-19 em cada estado. <br>
- caso_full.csv.gz: possuí informações completas de casos de COVID-19 em cada estado. <br>
- obito_cartorio.csv.gz: possuí o tipo e volume das causas de óbitos por COVID-19 em cada estado. <br>

### Tratamento dos dados

In [22]:
#Como o boletim não é pertinente para à análise, por ser somente a origem da fonte dos dados, será desconsiderado.

df_caso = pd.read_csv(lista_arquivos[1], compression='gzip')
df_caso_full = pd.read_csv(lista_arquivos[2], compression='gzip')
df_obito = pd.read_csv(lista_arquivos[3], compression='gzip')

#### df_caso/df_caso_full

In [52]:
#display(df_caso)
print(df_caso.iloc[0])
print(60*'-')
print(df_caso_full.iloc[0])

print(df_caso['date'][0])
print(df_caso['date'].tail(1))


print(df_caso_full['date'][0])
print(df_caso_full['date'].tail(1))

#Como as informações presentes no df_caso_full são mais completas, ele será utilizado para a análise em questão.
#Sendo assim, df_caso será desconsiderado.

date                               2022-03-27
state                                      AP
city                                      NaN
place_type                              state
confirmed                              160328
deaths                                   2122
order_for_place                           734
is_last                                  True
estimated_population_2019            845731.0
estimated_population                 861773.0
city_ibge_code                           16.0
confirmed_per_100k_inhabitants    18604.43527
death_rate                             0.0132
Name: 0, dtype: object
------------------------------------------------------------
city                                             Rio Branco
city_ibge_code                                    1200401.0
date                                             2020-03-17
epidemiological_week                                 202012
estimated_population                               413418.0
estimated_populati

#### df_caso_full

In [53]:
#is_repeated mostra que os dados estão sendo repetidos, portanto as linhas onde is_repeated = True devem ser excluídas
#para não se ter viéses na análise

df_caso_full = df_caso_full.drop(df_caso_full[df_caso_full['is_repeated'] == True].index)

display(df_caso_full)

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-17,0.0000,0,1,city,AC,3,0
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0000,0,1,state,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-18,0.0000,0,2,city,AC,0,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0000,0,2,state,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,False,4,0.96754,2020-03-19,0.0000,0,3,city,AC,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3852807,NaN,17.0,2022-03-21,202212,1590248.0,1572866.0,False,False,302166,19001.18724,2022-03-21,0.0137,4141,734,state,TO,300,1
3852947,NaN,17.0,2022-03-22,202212,1590248.0,1572866.0,False,False,302289,19008.92188,2022-03-22,0.0137,4141,735,state,TO,123,0
3853087,NaN,17.0,2022-03-23,202212,1590248.0,1572866.0,False,False,302387,19015.08444,2022-03-23,0.0137,4142,736,state,TO,98,1
3853227,NaN,17.0,2022-03-24,202212,1590248.0,1572866.0,False,False,302443,19018.60590,2022-03-24,0.0137,4142,737,state,TO,56,0


In [55]:
#Feito a exclusão das linhas, agora a coluna pode ser excluída por não ser mais pertinente à análise

df_caso_full = df_caso_full.drop('is_repeated', axis=1)



,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,3,0.72566,2020-03-17,0.0000,0,1,city,AC,3,0
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,3,0.33539,2020-03-17,0.0000,0,1,state,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,3,0.72566,2020-03-18,0.0000,0,2,city,AC,0,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,3,0.33539,2020-03-18,0.0000,0,2,state,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,4,0.96754,2020-03-19,0.0000,0,3,city,AC,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3852807,NaN,17.0,2022-03-21,202212,1590248.0,1572866.0,False,302166,19001.18724,2022-03-21,0.0137,4141,734,state,TO,300,1
3852947,NaN,17.0,2022-03-22,202212,1590248.0,1572866.0,False,302289,19008.92188,2022-03-22,0.0137,4141,735,state,TO,123,0
3853087,NaN,17.0,2022-03-23,202212,1590248.0,1572866.0,False,302387,19015.08444,2022-03-23,0.0137,4142,736,state,TO,98,1
3853227,NaN,17.0,2022-03-24,202212,1590248.0,1572866.0,False,302443,19018.60590,2022-03-24,0.0137,4142,737,state,TO,56,0


In [56]:
#Como a análise será feita por cidade, as linhas com os campos da coluna city que não estiverem preenchidos serão excluídas

df_caso_full = df_caso_full.dropna(subset=['city'])

display(df_caso_full)

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,3,0.72566,2020-03-17,0.0000,0,1,city,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,3,0.72566,2020-03-18,0.0000,0,2,city,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,4,0.96754,2020-03-19,0.0000,0,3,city,AC,1,0
6,Rio Branco,1200401.0,2020-03-20,202012,413418.0,407319.0,False,7,1.69320,2020-03-20,0.0000,0,4,city,AC,3,0
8,Rio Branco,1200401.0,2020-03-21,202012,413418.0,407319.0,False,11,2.66075,2020-03-21,0.0000,0,5,city,AC,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3838382,Tocantínia,1721109.0,2021-12-08,202149,7617.0,7545.0,True,1013,13299.19916,2021-12-08,0.0128,13,569,city,TO,13,0
3838383,Tupirama,1721257.0,2021-12-08,202149,1922.0,1891.0,True,223,11602.49740,2021-12-08,0.0090,2,565,city,TO,0,0
3838384,Tupiratins,1721307.0,2021-12-08,202149,2729.0,2671.0,True,199,7292.04837,2021-12-08,0.0151,3,565,city,TO,0,0
3838385,Wanderlândia,1722081.0,2021-12-08,202149,11734.0,11683.0,True,1433,12212.37430,2021-12-08,0.0147,21,590,city,TO,0,0


In [59]:
#Agora algumas colunas não trazem nenhuma informação pertinente para a análise, essas colunas são:

colunas_drop = ['city_ibge_code', 'is_last', 'last_available_date', 'place_type', 'estimated_population_2019']

#Data da coleta de dado é igual a data da última coleta do dado, portanto a coluna last_available_date será excluída
df_bool = df_caso_full['date'] == df_caso_full['last_available_date']
print(df_bool.value_counts())

for coluna in colunas_drop:
    df_caso_full = df_caso_full.drop(coluna, axis=1)
    
display(df_caso_full)

True    2818241
dtype: int64


,city,date,epidemiological_week,estimated_population,estimated_population_2019,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_death_rate,last_available_deaths,order_for_place,state,new_confirmed,new_deaths
0,Rio Branco,2020-03-17,202012,413418.0,407319.0,3,0.72566,0.0000,0,1,AC,3,0
2,Rio Branco,2020-03-18,202012,413418.0,407319.0,3,0.72566,0.0000,0,2,AC,0,0
4,Rio Branco,2020-03-19,202012,413418.0,407319.0,4,0.96754,0.0000,0,3,AC,1,0
6,Rio Branco,2020-03-20,202012,413418.0,407319.0,7,1.69320,0.0000,0,4,AC,3,0
8,Rio Branco,2020-03-21,202012,413418.0,407319.0,11,2.66075,0.0000,0,5,AC,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3838382,Tocantínia,2021-12-08,202149,7617.0,7545.0,1013,13299.19916,0.0128,13,569,TO,13,0
3838383,Tupirama,2021-12-08,202149,1922.0,1891.0,223,11602.49740,0.0090,2,565,TO,0,0
3838384,Tupiratins,2021-12-08,202149,2729.0,2671.0,199,7292.04837,0.0151,3,565,TO,0,0
3838385,Wanderlândia,2021-12-08,202149,11734.0,11683.0,1433,12212.37430,0.0147,21,590,TO,0,0


#### Assim o df_caso_full está tratado e pronto para análises

No caso o arquivo tratado será salvo com um novo nome para quaisquer análises posteriores.

In [62]:
df_caso_full.to_csv('novo_caso_full.csv.gz', sep=',', index=False, compression='gzip')

#### df_obito

In [43]:
display(df_obito)
display(df_obito.groupby(['state']).sum())

,date,state,epidemiological_week_2019,epidemiological_week_2020,deaths_indeterminate_2019,deaths_respiratory_failure_2019,deaths_others_2019,deaths_pneumonia_2019,deaths_septicemia_2019,deaths_sars_2019,...,new_deaths_sars_2019,new_deaths_covid19,new_deaths_indeterminate_2020,new_deaths_respiratory_failure_2020,new_deaths_others_2020,new_deaths_pneumonia_2020,new_deaths_septicemia_2020,new_deaths_sars_2020,new_deaths_total_2019,new_deaths_total_2020
0,2020-01-01,AC,1,1,0,2,4,3,1,0,...,NaN,NaN,1.0,1.0,8.0,2.0,2.0,NaN,10,14
1,2020-01-02,AC,1,1,0,2,10,3,2,0,...,NaN,NaN,NaN,NaN,6.0,2.0,2.0,NaN,7,10
2,2020-01-03,AC,1,1,0,2,16,3,4,0,...,NaN,NaN,NaN,NaN,8.0,3.0,2.0,NaN,8,13
3,2020-01-04,AC,1,1,0,2,18,6,5,0,...,NaN,NaN,NaN,1.0,4.0,NaN,1.0,NaN,6,6
4,2020-01-05,AC,1,2,0,2,24,7,5,0,...,NaN,NaN,NaN,1.0,4.0,4.0,NaN,NaN,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9877,2020-12-27,TO,52,53,48,450,3120,889,511,7,...,NaN,3.0,NaN,NaN,1.0,NaN,NaN,NaN,14,4
9878,2020-12-28,TO,52,53,48,452,3127,891,512,7,...,NaN,1.0,NaN,NaN,4.0,1.0,NaN,NaN,12,6
9879,2020-12-29,TO,1,53,48,452,3135,895,514,8,...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,15,1
9880,2020-12-30,TO,1,53,48,455,3145,897,515,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,0


,epidemiological_week_2019,epidemiological_week_2020,deaths_indeterminate_2019,deaths_respiratory_failure_2019,deaths_others_2019,deaths_pneumonia_2019,deaths_septicemia_2019,deaths_sars_2019,deaths_covid19,deaths_indeterminate_2020,...,new_deaths_sars_2019,new_deaths_covid19,new_deaths_indeterminate_2020,new_deaths_respiratory_failure_2020,new_deaths_others_2020,new_deaths_pneumonia_2020,new_deaths_septicemia_2020,new_deaths_sars_2020,new_deaths_total_2019,new_deaths_total_2020
state,,,,,,,,,,,,,,,,,,,,,
AC,9656,9908,7023,42879,360091,160954,76979,1050,128272,7408,...,5.0,789.0,39.0,231.0,2081.0,631.0,404.0,29.0,3568,4204
AL,9656,9908,17586,286545,1817912,478329,383073,2454,409503,18231,...,13.0,2340.0,100.0,1362.0,10643.0,1917.0,2146.0,315.0,16544,18823
AM,9656,9908,25720,123600,1536557,394030,341219,8421,499497,31391,...,35.0,2829.0,140.0,929.0,8323.0,2311.0,1837.0,764.0,13158,17133
AP,9656,9908,2197,43751,249320,91520,77937,863,140834,1076,...,7.0,762.0,6.0,240.0,1096.0,282.0,277.0,20.0,2462,2683
BA,9656,9908,22538,1403498,7926376,1344188,1695864,18530,1225750,30085,...,100.0,8421.0,140.0,6860.0,46055.0,6209.0,8765.0,468.0,67862,76918
CE,9656,9908,20754,545649,4414745,1374208,891899,8015,1840282,32698,...,39.0,9710.0,184.0,3166.0,26001.0,6053.0,4748.0,686.0,39143,50548
DF,9656,9908,23593,181607,1300573,585313,445125,6658,540671,21947,...,41.0,3939.0,140.0,966.0,7481.0,2774.0,2134.0,67.0,14053,17501
ES,9656,9908,2256,256920,2363681,938407,689298,6585,719106,846,...,34.0,5229.0,4.0,1376.0,12887.0,3828.0,3411.0,341.0,23302,27076
GO,9656,9908,25221,586965,2920373,1237158,715615,8843,935456,31806,...,45.0,7115.0,172.0,3324.0,15798.0,5684.0,3994.0,206.0,30272,36293
